In [10]:
import os, openai, pdfplumber, pprint
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from itertools import chain
from dotenv import load_dotenv

In [4]:
load_dotenv('../.env')
openai.api_key = os.getenv("CHATGPT-RECEIPT")

In [15]:
# def load_and_split_pdf(pdf_path):
#     pdf_loader = PyPDFLoader(pdf_path)
#     pdf_text = pdf_loader.load()
#     splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
#     return splitter.split_documents(pdf_text)

def extract_text_with_pdfplumber(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다 (pdfplumber 사용).
    Args:
        pdf_path (str): PDF 파일 경로.
    Returns:
        str: PDF에서 추출한 텍스트.
    """
    try:
        with pdfplumber.open(pdf_path) as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text()
        return text
    except Exception as e:
        print(f"PDF 텍스트 추출 중 오류 발생: {e}")
        return ""


def chunk_text(text, chunk_size=1000, chunk_overlap=100):
    """
    긴 텍스트를 청크로 나눕니다.
    Args:
        text (str): 입력 텍스트.
        chunk_size (int): 각 청크의 최대 길이.
        chunk_overlap (int): 청크 간의 겹치는 부분 길이.
    Returns:
        list: 청크 리스트.
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.split_text(text)
    return chunks


def embed_and_store(chunks, vectorstore_path="faiss_index"):
    """
    텍스트 청크를 벡터화하고 FAISS에 저장합니다.
    Args:
        chunks (list): 텍스트 청크 리스트.
        vectorstore_path (str): FAISS 저장소 경로.
    Returns:
        FAISS: 생성된 벡터 저장소.
    """
    try:
        embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
        vectorstore = FAISS.from_texts(chunks, embeddings)
        vectorstore.save_local(vectorstore_path)
        print(f"FAISS 저장소가 '{vectorstore_path}'에 저장되었습니다.")
        return vectorstore
    except Exception as e:
        print(f"임베딩 또는 FAISS 저장 중 오류 발생: {e}")
        return None


def build_retrieval_qa(vectorstore):
    '''ArithmeticError
    벡터 저장소를 사용하여 검색 기반 질의 응답 모델을 생성합니다.
    Args:
        vectorstore (FAISS): 벡터 저장소 객체
    Returns:
        RetrievalQA: 검색 기반 질의 응답 모델 객체
    '''
    retriever = vectorstore.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(
        llm = ChatOpenAI(model = 'gpt-4', openai_api_key = openai.api_key),
        retriever = retriever,
        return_source_documents = True
    )
    return qa_chain


#### 예제

#### FAISS Vector DB
* 너무 시간이 오래 걸림
* 메모리도 너무 많이 차지함

In [31]:
pdf_path = ["grant_guideline.pdf", "grant_usecase.pdf", "r-d-innovation.pdf"]
pdf_text = [extract_text_with_pdfplumber(f'../pdf/{x}') for x in pdf_path]
chunks = list(chain.from_iterable(pdf_text))
print(f"총 {len(chunks)}개의 청크로 나누었습니다.")
len(chunks)
# 텍스트 청크를 임베딩하고 FAISS 저장소에 저장
vectorstore = embed_and_store(chunks, vectorstore_path="faiss_index")

##### GPT없이 similarity만으로 검색

In [8]:
def load_vectorstore(vectorstore_path="faiss_index"):
    """
    로컬 FAISS 저장소를 로드합니다.
    Args:
        vectorstore_path (str): FAISS 저장소 경로.
    Returns:
        FAISS: 로드된 벡터 저장소.
    """
    try:
        return FAISS.load_local(vectorstore_path, OpenAIEmbeddings(openai_api_key=openai.api_key))
    except Exception as e:
        print(f"FAISS 저장소 로드 중 오류 발생: {e}")
        return None


def query_vectorstore(query, vectorstore):
    """
    FAISS 저장소를 쿼리하고 관련 텍스트를 검색합니다.
    Args:
        query (str): 사용자 질문.
        vectorstore (FAISS): 벡터 저장소.
    Returns:
        list: 검색된 텍스트 결과.
    """
    try:
        retriever = vectorstore.as_retriever()
        results = retriever.get_relevant_documents(query)
        return [result.page_content for result in results]
    except Exception as e:
        print(f"검색 중 오류 발생: {e}")
        return []

In [9]:
query = "연구비로 가구 구매가 가능해?"
results = query_vectorstore(query, vectorstore)
print("\n검색 결과:")
for idx, result in enumerate(results):
    print(f"{idx + 1}: {result}")

/var/folders/39/jrqp49_54fj1tl_h3pjv45780000gn/T/ipykernel_90963/4274326300.py:27: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)



검색 결과:
1: 사무용기기 및 사무용 ㆍ최종(단계)종료 2개월 전까지
➡ ㆍ최종(단계)종료 전까지
소프트웨어 구입기한 (개인용PC는 비영리기관만 허용)
Q30. 실험실 책상 수리비?
실험실 내 학생의 책상 수리를 진행하고자 합니다. 연구실운영비(연구환경유지비)로 집행하면 될 지 확인을
부탁드립니다.
A. 가능합니다. 단, 자체규정을 확인해야 합니다.
정부연구개발비(직접비)는 수행연구과제와 직접적인 관련이 있고 연구목적 달성에 필요한 경우 사용합니다.
문의하신 실험실 책상 수리가 이에 부합하는지 검토할 필요가 있습니다. 국가연구개발사업 연구개발비 사용기준
제25조(연구활동비 공통 사용기준) ⑩항에 따르면 연구실운영비는 연구개발기관 자체규정에 따라 사용하여야
합니다. 따라서 문의내용은 관련 내용을 종합적으로 검토하여 판단해야 할 것으로 사료됩니다.
Q31. 연구근접지원인력 출장비 사용?
수행연구과제의 연구행정을 맡고 있는 연구근접지원인력에 대한 출장비 사용이 가능한가요?
A. 가능합니다.
종전에는 연구지원인력에 대해 출장비 등 연구활동비 사용비 불가했으나, 혁신법에서는 연구과제와의 직접적인
관련이 있는 경우 연구활동비(출장비 등) 사용이 가능합니다.
53정부연구개발비 사용 Q&A 사례집
Q32. 내부참석자만의 회의비 처리?
인문사회분야 학술지원사업에서 내부 소속 인력만 참석하는 회의비(식비) 사용이 가능한가요?
A. 불가합니다.
국가연구개발혁신법에서 수행하는 연구개발과제는 해당 연구개발기관에 소속된 자만 참여하는 회의에 대하여는
회의비 중 식비를 사용할 수 없습니다. 해당 연구개발기관이 아닌 다른 곳에 소속된 자가 1인 이상 참석해야
합니다(국가연구개발사업연구개발비 사용기준 제25조(연구활동비 공통 사용기준).
<국가연구개발사업 연구개발비 사용 기준 제25조(연구활동비 공통 사용기준)
③ 연구개발기관의 장은 해당 연구개발기관에 소속된 자만 참여하는 회의에 대하여는 회의비 중 식비를 계상하여서는
아니 된다. 다만, 정부출연기관의 기본사업에 대해서는 계상할 수 있

##### GPT를 활용한 응답

In [13]:
qa_system = build_retrieval_qa(vectorstore)

/var/folders/39/jrqp49_54fj1tl_h3pjv45780000gn/T/ipykernel_90963/2522637370.py:71: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model = 'gpt-4', openai_api_key = openai.api_key),


In [24]:
query = "국가연구개발혁신법 24조 내용을 말해줘."
results = qa_system({"query": query})
print(f"검색 결과:{results}")

검색 결과:{'query': '국가연구개발혁신법 24조 내용을 말해줘.', 'result': '죄송합니다, 제공된 정보 내에 국가연구개발혁신법 제24조의 구체적인 내용이 들어있지 않습니다. 해당 법률의 내용은 관련 법률 자료를 통해 확인하실 수 있습니다.', 'source_documents': [Document(id='eeb7f33c-861b-4a2a-b067-adfa4fd6a5e8', metadata={}, page_content='관련 국가연구개발혁신법 시행령 제24조(연구개발비의 관리), 국가연구개발사업 연구개발비 사용 기준 제22조\n법규 (연구개발비 공통 인정기준) ⑤항\n⑧ 연구개발비는 객관적인 증명서류(전자증빙 등)를 구비하여 집행한다.\n- (증명자료) 결의서, 영수증서(카드매출전표, 계좌이체 확인서, 세금계산서 등) 등\n- 증명자료는 연구개발기간(단계) 종료 후 5년동안 보관해야 하며, 「전자문서 및 전자거래\n기본법」등에 따른 전자문서, 전자화문서로 보관가능(다만, 다른 법령에 특별한 규정이 있는\n경우에는 그 법령의 해당 규정에 따름)\n- (연구개발비 사용내역 입력) 연구개발비통합정보시스템(www.gaia.go.kr)에 등록\n관련 국가연구개발혁신법 제24조(연구개발비의 관리), 국가연구개발사업 연구개발비 사용 기준 제22조(연구개발비\n법규 공통 인정기준) ②항 및 제72조(연구개발비 사용 증명자료의 보관)\n5정부연구개발비 사용 Q&A 사례집\n이해충돌 회피\n⑨ 연구개발과제의 인건비와 연구수당은 참여연구자별 인건비 계상률과\n기여도에 따라 합리적이고 공정하게 지급하여야 한다.\n- 인건비나 연구수당 지급액과 관련된 당사자들 간에 갈등이 발생하지 않도록 당초 연구개발\n계획에 따라 연구를 수행하되, 계획 변경이 필요한 경우는 이해 당사자들에게 변경사항을\n문서를 통해 알려야 한다.\n관련 국가연구개발사업 연구개발비 사용 기준 제26조(연구수당 공통 사용기준) ④항\n법규\n➉ 연구개발비 집행은 공무(公務)이므로 사무(私務)와 명확히 

In [25]:
pprint.pprint(results['result'])

('죄송합니다, 제공된 정보 내에 국가연구개발혁신법 제24조의 구체적인 내용이 들어있지 않습니다. 해당 법률의 내용은 관련 법률 자료를 통해 '
 '확인하실 수 있습니다.')


#### GPT Assistant를 사용

In [7]:
loader= PyPDFLoader("../pdf/KLM Quantum-textbook.pdf")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
vector_store = FAISS.from_documents(texts, embeddings)

In [ ]:
pdf_text = extract_text_with_pdfplumber('../pdf/r-d-innovation.pdf')
texts = chunk_text(pdf_text)
embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
vector_store = FAISS.from_texts(texts, embeddings)

In [49]:
# Vector Store에서 검색
def search_documents(query):
    retriever = vector_store.as_retriever()
    docs = retriever.get_relevant_documents(query, k = 4)
    return docs

# Assistant + Vector Store    
def call_assistant_with_docs(user_input):
    # 파일 검색 결과 가져오기
    relevant_docs = search_documents(user_input)
    relevant_texts = "\n\n".join([doc.page_content for doc in relevant_docs])
    print(f"검색 결과: {len(relevant_docs)}개의 문서에서 발견됨.")
    # print(f'relevant_texts: {relevant_texts}')
    messages = []
    # 대화 시작 시 시스템 지침 추가 (한 번만 추가)
    if len(messages) == 0:
        messages.append(
            {
                "role": "system", 
                "content": """
                넌 국가연구개발 혁신법에 대해 잘 아는 행정 관련 전문가야.
                만약 Q&A에 해당 내용이 있으면 이에 대해 자세하고 친절하게 답변해줘.
                정확하다고 생각되는 답을 찾았으면, 가장 먼저 "예" 혹은 "아니오"로 답변을 시작하고, 그렇지 않으면 "모르겠어"라고 말해줘.
                답한 내용의 근거를 정확하게 말해주되 문서내 검색한 내용만 답해 줘.
                """
            }
        )
    
    # 사용자 입력과 검색된 문서를 추가
    messages.append({"role": "user", "content": user_input})
    if relevant_texts.strip():  # 검색 결과가 있을 경우에만 추가
        messages.append({"role": "system", "content": f"다음 문서를 참고하세요:\n{relevant_texts}"})
    
    # OpenAI API 호출
    response = openai.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=messages,
        max_tokens=500,
        temperature=0.7,
    )
    
    # 모델 응답
    assistant_response = response.choices[0].message.content
    messages.append({"role": "assistant", "content": assistant_response})
    return assistant_response

In [51]:
call_assistant_with_docs("공동연구기관 변경을 하기 위해서는 어떤 절차를 밟아야 하나요?")

검색 결과: 4개의 문서에서 발견됨.


'예, 공동연구기관 변경을 하기 위해서는 다음과 같은 절차를 밟아야 합니다:\n\n1. 변경 사유와 내용을 중앙행정기관의 장에게 제출합니다.\n2. 필요한 경우, 중앙행정기관의 장은 보완을 요구할 수 있습니다.\n3. 중앙행정기관의 장은 연구개발비 사용 계획의 변경 승인 여부를 사용 계획 변경 신청 접수 후 15일 이내에 연구개발기관의 장에게 통보해야 합니다. 변경 승인 여부를 결정하기 어려운 경우 기간을 연장할 수 있으며, 연장한 기간을 연구개발기관의 장에게 통보해야 합니다.\n4. 부처, 전문기관 및 연구개발기관은 상호간에 문서로 사전에 협의해야 합니다. 이는 협약변경 내용과 사유를 담은 공문을 발송하거나 통합정보 시스템을 통해 당사자 간 사전에 협의를 진행하는 것을 포함합니다.\n5. 특별평가를 거쳐 연구개발과제의 목표, 연구책임자 등을 변경하거나 연구개발과제를 중단할 수 있는 경우도 있습니다.\n\n위 내용은 국가연구개발혁신법에 관한 문서에서 확인할 수 있습니다.'

In [ ]:
'예, 공동연구기관 변경을 하기 위해서는 다음과 같은 절차를 밟아야 합니다:\n\n
1. 변경 사유와 내용을 중앙행정기관의 장에게 제출합니다.\n
2. 필요한 경우, 중앙행정기관의 장은 보완을 요구할 수 있습니다.\n
3. 중앙행정기관의 장은 연구개발비 사용 계획의 변경 승인 여부를 사용 계획 변경 신청 접수 후 15일 이내에 연구개발기관의 장에게 통보해야 합니다. 변경 승인 여부를 결정하기 어려운 경우 기간을 연장할 수 있으며, 연장한 기간을 연구개발기관의 장에게 통보해야 합니다.\n
4. 부처, 전문기관 및 연구개발기관은 상호간에 문서로 사전에 협의해야 합니다. 이는 협약변경 내용과 사유를 담은 공문을 발송하거나 통합정보 시스템을 통해 당사자 간 사전에 협의를 진행하는 것을 포함합니다.\n
5. 특별평가를 거쳐 연구개발과제의 목표, 연구책임자 등을 변경하거나 연구개발과제를 중단할 수 있는 경우도 있습니다.\n\n
위 내용은 국가연구개발혁신법에 관한 문서에서 확인할 수 있습니다.'

In [52]:
call_assistant_with_docs("변경할 내용에 대해 자세한 자료 및 근거는 어떤 것들을 준비해야 해? 양식이 따로 있나?")

검색 결과: 4개의 문서에서 발견됨.


'예, 변경할 내용에 대해 자세한 자료 및 근거를 준비해야 하는 경우, 부처별 자체지침을 우선 적용하며, 가이드라인을 참고하여야 합니다. 특히, 국가연구개발혁신법 시행령 제31조에 따르면, 특별평가를 실시할 경우 연구개발기관 또는 연구책임자에게 특별평가 실시 시기, 사유, 소명 자료의 제출 시한 등을 통보해야 하며, 변경이나 중단을 요청할 때는 과학기술정보통신부령으로 정하는 요청서에 해당 사유를 증명하는 서류를 첨부하여 제출해야 합니다.\n\n협약 변경 시에는 상호간 문서를 통한 사전 협의 또는 특별평가를 명시하고 있으며, 연구개발환경의 변경 등으로 연구개발과제를 계속 수행하는 것이 불필요하다고 판단되어 특별평가를 거쳐 연구개발과제를 중단하는 경우 최종평가가 실시되어야 함을 규정하고 있습니다.\n\n따라서 변경할 내용에 대한 자세한 자료 및 근거 준비 시 해당 부처의 자체지침을 참조하고, 필요한 경우 특별평가 요청서 및 증명 서류를 준비해야 합니다. 양식은 부처 또는 관련 법령에 따라 정해질 수 있으므로, 구체적인 양식'

In [53]:
call_assistant_with_docs("내가 앞에서 물어본 변경이 무슨 변경이야?")

검색 결과: 4개의 문서에서 발견됨.


'모르겠어'